# Transpose Lower Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := L^T x $ where $ L $ is lower triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think, especially when you work with upper triangular matrix $ U $, later.  You may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

## The `Trmv_lt_unb_var1!( L, x )` routine

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L^T x $.  The "_lt_" in the name of the routine indicates this is the "lower, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_lt_unb_var1!(L, x)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    while size(LTL, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.scal!( lambda11, chi1 )
        laff.dots!( l21, x2, chi1 )
        
        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_lt_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
L = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.556903   0.548677  0.619914  0.0196764
 0.62574    0.778596  0.187776  0.765051 
 0.0781279  0.646267  0.714715  0.301121 
 0.897779   0.124974  0.673278  0.826303 

In [3]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.9278287266579546 
 0.01307692304486241
 0.9277551358656992 
 0.9692985092442064 

In [4]:
laff.copy!( x, xold )   # save the original vector x

Trmv_lt_unb_var1!( L, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 1.4675921922449449
 0.7308963507303995
 1.3156880225545486
 0.8009343259995998

In [5]:
# LowerTriangular makes the matrix lower triangular
using LinearAlgebra
println("x - ( transpose( LowerTriangular( L ) ) * xold )")
x - ( transpose( LowerTriangular( L ) ) * xold )

x - ( transpose( LowerTriangular( L ) ) * xold )


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The `Trmv_lt_unb_var2!( L, x )` routine

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L^T x $.  The "_lt_" in the name of the routine indicates this is the "lower triangular, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [6]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmv_lt_unb_var2!(L, x)

    LTL, LTR, 
    LBL, LBR  = flame.part_2x2(L, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    while size(LTL, 1) < size(L, 1)

        L00,  l01,      L02,  
        l10t, lambda11, l12t, 
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, 
                                                        LBL, LBR, 
                                                        1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, l10t, x0 )
        laff.scal!( lambda11, chi1 )
        
        #------------------------------------------------------------#

        LTL, LTR, 
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  
                                               l10t, lambda11, l12t, 
                                               L20,  l21,      L22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmv_lt_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [7]:
L = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that L is not lower triangular.  We will only use the lower triangular part.

println( "L before =" )
L

L before =


4×4 Array{Float64,2}:
 0.563549  0.0701596  0.425021  0.822183 
 0.997585  0.263739   0.719536  0.131133 
 0.866218  0.585092   0.692155  0.932923 
 0.362787  0.399352   0.367231  0.0227959

In [8]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.2910842973437695 
 0.21462706264249753
 0.3294539787510893 
 0.5702917868665041 

In [9]:
laff.copy!( x, xold )   # save the original vector x

Trmv_lt_unb_var2!( L, x )

println( "x after =" )
x

x after =


4-element Array{Float64,1}:
 0.8704226567769852  
 0.4771132648883356  
 0.43746235550468526 
 0.013000293924807152

In [10]:
# LowerTriangular makes the matrix lower triangular
using LinearAlgebra
println("x - ( transpose( LowerTriangular( L ) ) * xold )")
x - ( transpose( LowerTriangular( L ) ) * xold )

x - ( transpose( LowerTriangular( L ) ) * xold )


4-element Array{Float64,1}:
 -1.1102230246251565e-16
  0.0                   
  0.0                   
  0.0                   

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.